In [3]:
import AlgoCrac as ac
import numpy as np
import pandas as pd
import yfinance as yf
from abc import ABC, abstractmethod
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from functools import partial
import matplotlib.pyplot as plt
import datetime
import time
#from indicators import moving_average, exp_moving_average, vwap
#from Signals import *
from AlgoCrac import moving_average, exp_moving_average, vwap
from AlgoCrac.Signals.Signals import *
from AlgoCrac.Signals.Signals_space import intraday_space, space, intraday_space_mini
from AlgoCrac.Signals.utils import *
from AlgoCrac.Strategy.Strategy import *


from tqdm import tqdm

In [4]:
timeframe = "5m"
sym = ['WIX','MMM','GOOGL','TSLA','TEVA']

#sym = ["BDN",'MMM']
#sym = ['MMM']
data = ac.get_data(sym, start = "2024-06-10",end="2024-06-13",interval = timeframe)

#data = ac.get_nasdaq100_data(start = "2024-06-1",end="2024-06-13",interval = timeframe)

[*********************100%***********************]  5 of 5 completed


In [5]:


def gen_signals(space, prob = {"Crossover": 0.5}):
    params = {}
    space_signals = list(space.keys())

    # check if the keys of prob are in space
    for signal_type in prob.keys():
        if signal_type not in space_signals:
            raise ValueError("The signal type {} is not in the space".format(signal_type))
        
    # check if the sum of the probabilities is equal to 1
    prob_sum = np.sum(list(prob.values()))
    if prob_sum > 1:
        raise ValueError("The sum of the probabilities is not lower or equal to 1")
       
    if prob!={} or prob is not None:
        p = np.zeros(len(space))
        for i, signal_type in enumerate(space_signals):
            if signal_type in prob.keys():
                p[i] = prob[signal_type]
            else:
                p[i] = (1-prob_sum)/(len(space)-len(prob))
    # equal probabilities
    else:
        p = np.ones(len(space))/len(space)


    # generate the signal
    signal_type = np.random.choice(space_signals, p = p)
    space_signal = copy.deepcopy(space[signal_type])
    signal_class = space_signal['class']
    params = space_signal['params']

    # generate the parameters
    for p in params:
        params[p] = np.random.choice(params[p])
    gen_signal = signal_class(**params)
    
    return gen_signal



gen_signals(intraday_space).profile

{'class': 'Range',
 'params': {'max_val': 25,
  'min_val': 425,
  'metric': 'first',
  'in_range': False,
  'on': 'Volume',
  'invert': True,
  'period': None,
  'min_timesteps': None,
  'min_lookback': 3,
  'cancel_first_k': 1}}

In [6]:
def gen_strategy(space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 5, hard_entry_signals = [], hard_exit_signals = [], exit_signals = False):

    max_entry_signals = np.random.randint(1,max_signals+1)
    entry_signals = copy.deepcopy(hard_entry_signals)
    for i in range(max_entry_signals):
        entry_signals.append(gen_signals(space))
    entry_signal = Confluence(signals_list = entry_signals, max_signals =max_entry_signals, invert = False)

    if exit_signals == True:
        max_exit_signals = np.random.randint(1,max_signals+1)
        exit_signals = copy.deepcopy(hard_exit_signals)
        for i in range(max_exit_signals):
            exit_signals.append(gen_signals(space))
        exit_signal = Confluence(signals_list = exit_signals, max_signals = max_exit_signals, invert = False)
    else:
        exit_signal = Confluence(signals_list = copy.deepcopy(entry_signals), max_signals = max_entry_signals, invert = True)

    st = Strategy(entry_signal = entry_signal,
                exit_signal = exit_signal,
                timeframe = timeframe, 
                trade_allocation = trade_allocation,
                allocation = allocation,
                negative_allocation = negative_allocation,
                side = side)
    
    return st


timeframe = "5m"
trade_allocation = 1000
allocation = 10000
negative_allocation= False
side = "long"

st = gen_strategy(intraday_space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 1)
st = gen_strategy(intraday_space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 1)

print(st.entry_signal.signals_list)
st.entry_signal.signals_list[0].profile




{'class': 'Crossover',
 'params': {'period': 28,
  'indicator': 'SMA',
  'on': 'Open',
  'invert': True,
  'eps': 0.1,
  'min_timesteps': None,
  'min_lookback': 3,
  'cancel_first_k': 1}}

In [56]:
# create signal with close price below 20

below_20 = BasicMetric(condition_value = 20,
                        metric = "last",
                        condition = "lower",
                        on = "Close", # Open, High, Low, Close, Volume
                        invert = False,
                        )

hard_entry_signals  = [below_20]
popsize = 10
generations = 2

st = gen_strategy(intraday_space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 1, hard_entry_signals = hard_entry_signals)

st.entry_signal.profile

{'class': 'Confluence',
 'signals_list': [{'class': 'BasicMetric',
   'params': {'condition_value': 20,
    'metric': 'last',
    'condition': 'lower',
    'on': 'Close',
    'invert': False,
    'period': None,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}},
  {'class': 'Crossover',
   'params': {'period': 24,
    'indicator': 'VWAP',
    'on': 'Close',
    'invert': True,
    'eps': 3.5,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}}]}

In [42]:
st.remove_signals([], entry = True)
st.insert_signals([], entry = True)

In [43]:
st.entry_signal.profile

{'class': 'Confluence',
 'signals_list': [{'class': 'BasicMetric',
   'params': {'condition_value': 20,
    'metric': 'last',
    'condition': 'lower',
    'on': 'Close',
    'invert': False,
    'period': None,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}},
  {'class': 'Crossover',
   'params': {'period': 17,
    'indicator': 'VWAP',
    'on': 'Open',
    'invert': True,
    'eps': 0.6000000000000001,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}}]}

In [8]:
def signal_mutate(signal, space):
    new_signal = copy.deepcopy(signal)
    signal_class = new_signal.profile['class']
    signal_params = new_signal.profile['params']

    signal_space = space[signal_class]
    signal_space_params = list(signal_space['params'].keys())

    select_param = np.random.choice(signal_space_params)
    new_value = np.random.choice(signal_space['params'][select_param])
    signal_params[select_param] = new_value
    
    return new_signal


def strategy_mutate(strategy, space, max_mutations = 2):

    strategy = copy.deepcopy(strategy)
    assert type(strategy.entry_signal) == Confluence
    assert type(strategy.exit_signal) == Confluence

    entry_signals = strategy.entry_signal.signals_list
    exit_signals = strategy.exit_signal.signals_list


    for i in range(max_mutations):
        if np.random.uniform() < 0.5:
            signal = np.random.choice(entry_signals)
            new_signal = signal_mutate(signal, space)
            entry_signals[entry_signals.index(signal)] = new_signal
        else:
            signal = np.random.choice(exit_signals)
            new_signal = signal_mutate(signal, space)
            exit_signals[exit_signals.index(signal)] = new_signal

    # mutate the entry signal
    new_entry_signal = Confluence(signals_list = entry_signals, max_signals = strategy.entry_signal.max_signals, invert = strategy.entry_signal.invert)
    new_exit_signal = Confluence(signals_list = exit_signals, max_signals = strategy.exit_signal.max_signals, invert = strategy.exit_signal.invert)

    # create the new strategy
    new_strategy = Strategy(entry_signal = new_entry_signal,
                            exit_signal = new_exit_signal,
                            timeframe = strategy.timeframe, 
                            trade_allocation = strategy.trade_allocation,
                            allocation = strategy.allocation,
                            negative_allocation = strategy.negative_allocation,
                            side = strategy.side)

    return new_strategy


new_st = strategy_mutate(st, intraday_space, max_mutations = 3)
print(new_st.entry_signal.signals_list)
new_st.entry_signal.signals_list[0].profile



[<AlgoCrac.Signals.Signals.BasicMetric object at 0x000001BD4D713250>, <AlgoCrac.Signals.Signals.RelativeSize object at 0x000001BD4D7106D0>]


{'class': 'BasicMetric',
 'params': {'condition_value': 20,
  'metric': 'last',
  'condition': 'lower',
  'on': 'Close',
  'invert': False,
  'period': None,
  'min_timesteps': None,
  'min_lookback': 3,
  'cancel_first_k': 1}}

In [12]:
def cross_strategy(strategy1, strategy2, max_signals = 5):
    assert type(strategy1.entry_signal) == Confluence
    assert type(strategy1.exit_signal) == Confluence
    assert type(strategy2.entry_signal) == Confluence
    assert type(strategy2.exit_signal) == Confluence

    strategy1 = copy.deepcopy(strategy1)
    strategy2 = copy.deepcopy(strategy2)

    entry_signals1 = strategy1.entry_signal.signals_list
    exit_signals1 = strategy1.exit_signal.signals_list

    entry_signals2 = strategy2.entry_signal.signals_list
    exit_signals2 = strategy2.exit_signal.signals_list



    # cross the entry signals
    all_entry_signals = entry_signals1 + entry_signals2
    n_signals = max_signals
    if n_signals >= len(all_entry_signals):
        n_signals = len(all_entry_signals)
    cross_entry_signals = np.random.choice(all_entry_signals, size = n_signals, replace = False).tolist()

    all_exit_signals = exit_signals1 + exit_signals2
    n_signals = max_signals
    if n_signals > len(all_exit_signals):
        n_signals = len(all_exit_signals)
    cross_exit_signals = np.random.choice(all_exit_signals, size = n_signals, replace = False).tolist()


    # create the new strategy
    entry_signal = Confluence(signals_list = cross_entry_signals, max_signals = strategy1.entry_signal.max_signals, invert = strategy1.entry_signal.invert)
    exit_signal = Confluence(signals_list = cross_exit_signals, max_signals = strategy1.exit_signal.max_signals, invert = strategy1.exit_signal.invert)

    new_strategy = Strategy(entry_signal = entry_signal,
                            exit_signal = exit_signal,
                            timeframe = strategy1.timeframe, 
                            trade_allocation = strategy1.trade_allocation,
                            allocation = strategy1.allocation,
                            negative_allocation = strategy1.negative_allocation,
                            side = strategy1.side)

    return new_strategy


st1 = gen_strategy(intraday_space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 2)
print(st1.entry_signal.signals_list)
st2 = gen_strategy(intraday_space, timeframe, trade_allocation,allocation,negative_allocation,side, max_signals = 2)
print(st2.entry_signal.signals_list)

new_st = cross_strategy(st1, st2, max_signals = 2)
print(new_st.entry_signal.signals_list)

[<AlgoCrac.Signals.Signals.Crossover object at 0x000001BD4D7135B0>, <AlgoCrac.Signals.Signals.Crossover object at 0x000001BD4D713E80>]
[<AlgoCrac.Signals.Signals.Crossover object at 0x000001BD4D76B880>, <AlgoCrac.Signals.Signals.Crossover object at 0x000001BD4D76A200>]


In [13]:
new_st 

In [11]:
def invert_strategy(strategy):
    inv_strategy = copy.deepcopy(strategy)
    assert type(inv_strategy.entry_signal) == Confluence
    assert type(inv_strategy.exit_signal) == Confluence

    inv_strategy.entry_signal.invert = not inv_strategy.entry_signal.invert
    inv_strategy.exit_signal.invert = not inv_strategy.exit_signal.invert

    return inv_strategy

new_st = invert_strategy(st)

In [63]:
a = {"Crossover": 0.5}
b = {"Crossover": 0.5}

a==b

True

In [68]:
st.entry_signal.profile

{'class': 'Confluence',
 'signals_list': [{'class': 'BasicMetric',
   'params': {'condition_value': 20,
    'metric': 'last',
    'condition': 'lower',
    'on': 'Close',
    'invert': False,
    'period': None,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}},
  {'class': 'Crossover',
   'params': {'period': 4,
    'indicator': 'VWAP',
    'on': 'High',
    'invert': True,
    'eps': 0.9,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}}]}

In [74]:

"""
def insert_signals(strategy, signals, entry = True):
    new_strategy = copy.deepcopy(strategy)
    signals = copy.deepcopy(signals)
    assert type(new_strategy.entry_signal) == Confluence
    assert type(new_strategy.exit_signal) == Confluence

    if type(signals) != list:
        signals = [signals]

    if entry == True:
        # check if signals with similar profile is already in the list, if so do not insert
        for s in signals:
            for s1 in new_strategy.entry_signal.signals_list:
                if s.profile == s1.profile:
                    return new_strategy
        new_strategy.entry_signal.signals_list += signals
    else:
        for s in signals:
            for s1 in new_strategy.exit_signal.signals_list:
                if s.profile == s1.profile:
                    return new_strategy
        new_strategy.exit_signal.signals_list += signals


    return new_strategy
"""

new_st = insert_signals(st, st.entry_signal.signals_list[0], entry = True)
new_st.entry_signal.profile

{'class': 'Confluence',
 'signals_list': [{'class': 'BasicMetric',
   'params': {'condition_value': 20,
    'metric': 'last',
    'condition': 'lower',
    'on': 'Close',
    'invert': False,
    'period': None,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}},
  {'class': 'Crossover',
   'params': {'period': 4,
    'indicator': 'VWAP',
    'on': 'High',
    'invert': True,
    'eps': 0.9,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}}]}

In [46]:
class EvoBacktest:
    """
    Backtest class is used to run a backtest on a given strategy and data

    Args:
        strategy (Strategy): Strategy object
        data (pd.DataFrame): Dataframe of the data to run the backtest on
        commission (float, optional): Commission rate per share. Defaults to 0.01.
        min_commission (float, optional): Minimum commission per trade. Defaults to 1.5.
        slipage (float, optional): Slippage per share. Defaults to 0.0.
        start_offset (int, optional): Number of rows to skip before starting the backtest. Defaults to 5.
        include_commission (bool, optional): Whether to include commission in the reward calculation. Defaults to True.
        fast (bool, optional): Whether to run the backtest in fast mode. Defaults to False. Note: fast mode defaults negative_allocation to True.
    """
    def __init__(self, data,
                 space,
                commission = 0.01,
                min_commission = 1.5,
                slipage = 0.0,
                start_offset = 5,
                include_commission = True,
                fast = False,
                hard_strategy_list = [],
                hard_entry_signals = [],
                hard_exit_signals = [],
                popsize = 100,
                generations = 10,
                intraday_generations = 10,
                frac_best = 0.1,
                frac_invert_worst = 0.1,
                frac_cross = 0.1,
                frac_mutate = 0.05,
                max_mutations = 2,
                seed = 0,
                timeframe = "5m",
                max_signals = 5,
                side = "long",
                trade_allocation = 1000,
                allocation = 10000,
                negative_allocation = False
                ):

        np.random.seed(seed)
        self.space = space
        self.hard_strategy_list = hard_strategy_list
        self.popsize = popsize
        self.generations = generations
        self.timeframe = timeframe
        self.max_signals = max_signals
        self.intraday_generations = intraday_generations
        self.hard_strategy_list = hard_strategy_list
        self.hard_entry_signals = hard_entry_signals
        self.hard_exit_signals = hard_exit_signals

        self.frac_best = frac_best
        self.frac_invert_worst = frac_invert_worst
        self.frac_cross = frac_cross
        self.frac_mutate = frac_mutate
        if self.frac_best + self.frac_invert_worst + self.frac_cross + self.frac_mutate >= 1:
            raise ValueError("The sum of the fractions is higher or equal to 1")
        self.frac_random = 1 - (self.frac_best + self.frac_invert_worst + self.frac_cross + self.frac_mutate)

        self.max_mutations = max_mutations
        
        self.strategy = None
        self.data = data
        self.commission = commission
        self.min_commission = min_commission
        self.slipage = slipage
        self.include_commission = include_commission
        self.fast = fast

        self.status = pd.DataFrame()
        self.status_closed = self.status.copy()
        self.start_offset = start_offset 
        self.side = side
        self.trade_allocation = trade_allocation
        self.allocation = allocation
        self.negative_allocation = negative_allocation

        
        strategy = None
        #super().__init__(strategy, data,  commission, min_commission, slipage, start_offset, include_commission, fast)


    def run(self, objective = 'sharpe ratio'):
        summary = pd.DataFrame(columns = ['total reward',
                                           'total commission',
                                             'avg reward', 'std reward',
                                               'sharpe ratio', '% reward',
                                                 'avg risk reward ratio',
                                                   'avg duration (min)',
                                                     'max drawdown',
                                                     'num trades',
                                                       'generations'])
        
        closed_trades = pd.DataFrame()

        # initialize the population
        strategy_list = self.hard_strategy_list
        hard_ids = np.arange(len(strategy_list))
        for i in range(self.popsize):
            strategy_list.append(gen_strategy(space = self.space,
                                            timeframe = self.timeframe,
                                            trade_allocation = self.trade_allocation,
                                            allocation = self.allocation,
                                            negative_allocation = self.negative_allocation,
                                            side = self.side,
                                            max_signals = self.max_signals,
                                            hard_entry_signals = self.hard_entry_signals,
                                            hard_exit_signals = self.hard_exit_signals))
            
        strategy_id = np.arange(len(strategy_list))
        # create dictionary with the strategy id and the strategy
        self.population = dict(zip(strategy_id, strategy_list))
        gen_pop_ids = np.array(list(self.population.keys()))
        self.next_id = len(gen_pop_ids)

        for gen_i in range(self.generations):   
            print(f"Generation {gen_i+1}/{self.generations}")
            # iterate over the population dictionary
            # tqdm is used to show a progress bar
            for strategy_id in tqdm(gen_pop_ids):
                # get the strategy
                strategy = self.population[strategy_id]
                strategy.reset()
               # print(strategy.entry_signal.profile )
               # print(strategy.exit_signal.profile)
                bt = Backtest(strategy, self.data, fast = self.fast, include_commission= self.include_commission)
                bt.run() 
                strategy_summary = bt.summary()
                strategy_summary = strategy_summary.loc['closed']

                # add the strategy id to the summary
                #strategy_summary['strategy_id'] = strategy_id
                if strategy_id not in summary.index:
                    summary.loc[strategy_id] = strategy_summary
                    summary.loc[strategy_id,'generations'] = 1
                else:
                    summary.loc[strategy_id,'generations'] += 1
                    summary.loc[strategy_id,'total reward'] += strategy_summary['total reward']
                    summary.loc[strategy_id,'num trades'] += strategy_summary['num trades']
                    summary.loc[strategy_id,'max drawdown'] = np.min([summary.loc[strategy_id]['max drawdown'], strategy_summary['max drawdown']])
                
                    old_num_trades = summary.loc[strategy_id,'num trades']
                    gen_num_trades = strategy_summary['num trades']
                    weighted_average = lambda x, y: (x*old_num_trades + y*gen_num_trades)/(old_num_trades + gen_num_trades) 
                    for col in ['total reward', 'total commission', 'avg reward', 'std reward', 'sharpe ratio', '% reward', 'avg risk reward ratio', 'avg duration (min)']:
                        if strategy_summary[col]!=0:   
                            summary.loc[strategy_id,col] = weighted_average(summary.loc[strategy_id][col], strategy_summary[col])
                    # fill na in summary.loc[strategy_id]
                    summary.loc[strategy_id].fillna(0, inplace = True)


                strategy_closed = strategy.closed_trades
                if len(strategy_closed)>0:
                    strategy_closed['strategy_id'] = strategy_id
                    strategy_closed['generation'] = gen_i+1
                    closed_trades = pd.concat([closed_trades, strategy_closed])

      
            gen_summary = summary.copy().loc[gen_pop_ids].sort_values(by = objective, ascending = False)


            positive_size = len(gen_summary[gen_summary['total reward']>0])
            positive_popsize = np.min([positive_size, self.popsize])


            # select the best strategies
            n_best = int(self.frac_best*positive_popsize)
            best_ids = gen_summary.head(n_best).index.values

            # invert the worst strategies
            n_invert = 0
            inv_ids = []
           # n_invert = int(self.frac_invert_worst*self.popsize)
           # worst_ids = gen_summary.tail(n_invert).index.values
           # inv_ids = []
           # for worst_id in worst_ids:
           #     self.population[self.next_id] = invert_strategy(self.population[worst_id])
           #     inv_ids.append(self.next_id)
           #     self.next_id += 1

            # cross the best strategies
            n_cross = int(self.frac_cross*positive_popsize)
            cross_ids = []
            for i in range(n_cross):
                strategy1 = self.population[np.random.choice(best_ids)]
                strategy2 = self.population[np.random.choice(best_ids)]
                self.population[self.next_id] = cross_strategy(strategy1, strategy2, max_signals = self.max_signals)
                self.population[self.next_id].insert_signals(self.hard_entry_signals, entry = True)
                self.population[self.next_id].insert_signals(self.hard_exit_signals, entry = False)
                cross_ids.append(self.next_id)
                self.next_id += 1

            # mutate the best strategies
            n_mutate = int(self.frac_mutate*positive_popsize)
            mutate_ids = []
            for i in range(n_mutate):
                strategy = self.population[np.random.choice(best_ids)]
                self.population[self.next_id] = strategy_mutate(strategy, self.space, self.max_mutations)
                self.population[self.next_id].insert_signals(self.hard_entry_signals, entry = True)
                self.population[self.next_id].insert_signals(self.hard_exit_signals, entry = False)
                mutate_ids.append(self.next_id)
                self.next_id += 1
                

            # generate random strategies
            n_random = self.popsize - (n_best + n_invert + n_cross + n_mutate)
            random_ids = []
            for i in range(n_random):
                self.population[self.next_id] = gen_strategy(space = self.space,
                                            timeframe = self.timeframe,
                                            trade_allocation = self.trade_allocation,
                                            allocation = self.allocation,
                                            negative_allocation = self.negative_allocation,
                                            side = self.side,
                                            max_signals = self.max_signals,
                                            hard_entry_signals = self.hard_entry_signals,
                                            hard_exit_signals = self.hard_exit_signals)
                random_ids.append(self.next_id)
                self.next_id += 1

            # hard strategies
            gen_pop_ids = np.concatenate([hard_ids, best_ids, inv_ids, cross_ids, mutate_ids, random_ids])
        return summary, closed_trades




In [47]:
# create signal with close price below 20

below_20 = BasicMetric(condition_value = 20,
                        metric = "last",
                        condition = "lower",
                        on = "Close", # Open, High, Low, Close, Volume
                        invert = False,
                        )

hard_entry_signals  = [below_20]
popsize = 10
generations = 2




eb = EvoBacktest(data = data,
                space = intraday_space,
                commission = 0.01,
                min_commission = 1.5,
                slipage = 0.0,
                start_offset = 5,
                include_commission = True,
                fast = True,
                hard_strategy_list = [],
                hard_entry_signals = hard_entry_signals,
                hard_exit_signals = [],
                popsize = popsize,
                generations = generations,
                intraday_generations = 10,
                frac_best = 0.1,
                frac_invert_worst = 0.1,
                frac_cross = 0.1,
                frac_mutate = 0.05,
                max_mutations = 2,
                seed = 2,
                timeframe = "5m",
                max_signals = 3,
                side = "long",
                trade_allocation = 1000,
                allocation = 10000,
                negative_allocation = False
                )

summary, closed_trades = eb.run()

Generation 1/2


100%|██████████| 10/10 [00:00<00:00, 11.20it/s]


Generation 2/2


100%|██████████| 10/10 [00:00<00:00, 13.61it/s]


In [53]:
eb.population[9].entry_signal.profile

{'class': 'Confluence',
 'signals_list': [{'class': 'BasicMetric',
   'params': {'condition_value': 20,
    'metric': 'last',
    'condition': 'lower',
    'on': 'Close',
    'invert': False,
    'period': None,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}},
  {'class': 'Crossover',
   'params': {'period': 29,
    'indicator': 'EMA',
    'on': 'Volume',
    'invert': True,
    'eps': 0.0,
    'min_timesteps': None,
    'min_lookback': 3,
    'cancel_first_k': 1}}]}

In [55]:
closed_trades

,entry time,entry date,exit time,exit date,entry price,exit price,symbol,reward,position,commission,market value,id,strategy_id,generation
0,09:40:00,2024-06-10,15:55:00,2024-06-10,173.440002,175.009995,GOOGL,7.849960,5,1.5,875.049973,0,0.0,1
1,09:40:00,2024-06-10,15:55:00,2024-06-10,100.489998,100.855003,MMM,3.285049,9,1.5,907.695030,1,0.0,1
2,09:40:00,2024-06-10,15:55:00,2024-06-10,16.684999,16.834999,TEVA,8.849977,59,1.5,993.264946,2,0.0,1
3,09:40:00,2024-06-10,15:55:00,2024-06-10,176.744598,174.029999,TSLA,-13.572998,5,1.5,870.149994,3,0.0,1
4,09:40:00,2024-06-10,15:55:00,2024-06-10,170.130005,169.389999,WIX,-3.700027,5,1.5,846.949997,4,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19,09:40:00,2024-06-12,12:00:00,2024-06-12,171.815002,169.539993,WIX,-11.375046,5,1.5,847.699966,19,19.0,2
20,12:05:00,2024-06-12,12:50:00,2024-06-12,169.419998,170.389999,WIX,4.850006,5,1.5,851.949997,20,19.0,2
21,12:55:00,2024-06-12,15:20:00,2024-06-12,170.380005,170.179993,WIX,-1.000061,5,1.5,850.899963,21,19.0,2
22,14:10:00,2024-06-12,15:55:00,2024-06-12,176.720001,177.550003,GOOGL,4.150009,5,1.5,887.750015,22,19.0,2


In [29]:
summary.sort_values(by = 'generations', ascending = False).head(10)




,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,generations
0.0,-15.355837,22.5,-1.023722,15.887397,-0.966542,-0.153558,1.000473,375.0,-0.583391,15,1
137.0,-192.204477,217.5,-1.325548,3.069093,-62.625814,-1.922045,1.000189,17.413793,-0.194648,145,1
127.0,0,0,0,0,0,0,0,0,0,0,1
128.0,-32.824518,22.5,-2.188301,4.389486,-7.477986,-0.328245,0.999231,80.0,-0.162787,15,1
129.0,0,0,0,0,0,0,0,0,0,0,1
130.0,0,0,0,0,0,0,0,0,0,0,1
131.0,0,0,0,0,0,0,0,0,0,0,1
132.0,0,0,0,0,0,0,0,0,0,0,1
133.0,0,0,0,0,0,0,0,0,0,0,1
134.0,0,0,0,0,0,0,0,0,0,0,1


In [30]:
summary.sort_values(by = 'sharpe ratio', ascending = False).head(10)

,total reward,total commission,avg reward,std reward,sharpe ratio,% reward,avg risk reward ratio,avg duration (min),max drawdown,num trades,generations
123.0,18.19412,22.5,1.212941,11.72854,1.551269,0.181941,1.002973,245.0,-0.422217,15,1
13.0,18.19412,22.5,1.212941,11.72854,1.551269,0.181941,1.002973,245.0,-0.422217,15,1
135.0,10.567902,22.5,0.704527,9.825228,1.075588,0.105679,1.002338,315.0,-0.406539,15,1
74.0,16.948164,15.0,1.694816,15.79362,1.073102,0.169482,1.003549,362.0,-0.583391,10,1
167.0,9.168036,9.0,1.528006,16.858479,0.543823,0.09168,1.003063,183.333333,-0.505799,6,1
46.0,13.663084,4.5,4.554361,29.368456,0.46523,0.136631,1.006087,375.0,-0.583391,3,1
92.0,13.663084,4.5,4.554361,29.368456,0.46523,0.136631,1.006087,375.0,-0.583391,3,1
57.0,0.500685,27.0,0.027816,12.926632,0.038733,0.005007,1.001651,251.111111,-0.583391,18,1
91.0,0,0,0,0,0,0,0,0,0,0,1
90.0,0,0,0,0,0,0,0,0,0,0,1
